In [1]:

# # Project Overview
# 
# 
# <img src="images/pld3280-fig-0001-m.jpg" width="800" height="600">


# # Testing the various thresholding methods
# 
# ## Otsu thresholding, post processing, and display
# ## vs Adaptive thresholding, post processing, and display 
# ## vs Adaptive Gaussian thresholding, post processing, and display
# 


import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import pandas
import glob
from itertools import product

def get_isolated_membrane(image, method, blur=True, equalize=True, metadata=None, th_metadata=None):
    
    if metadata is None:
        raise ValueError("Metadata must be provided")
    
    if th_metadata is None:
        raise ValueError("Thresholding metadata must be provided")
    
    processed = process_image(image, method, blur=blur, equalize=equalize, metadata=metadata, th_metadata=th_metadata[method])
    return processed["overlayed"]

def load_gray_image(image_path):
    print(f"Loading image from {image_path}")
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return gray

def load_images_from_metadata(metadata_path, image_dir):
    """
    roi_metadata_path = "./metadata/rois_metadata_bignine.csv"
    image_dir = "rois
    """
    all_images = []
    
    # load the df and print the first few rows
    metadata = pandas.read_csv(metadata_path)
    
    # if the df is empty, return an empty, throw
    if len(metadata) == 0:
        raise ValueError(f"No metadata found in {metadata_path}")
    else:
        print(f"Loaded metadata from {metadata_path}, with {len(metadata)} rows")
    
    for image_number in len(metadata):
        image = load_gray_image(images[image_number])
        all_images.extend(image)
    
    return all_images

def equalize_image(image, clip_low=100, clip_high=255):
    equalized = cv2.equalizeHist(image)
    equalized = np.clip(equalized, clip_low, clip_high)
    return equalized

def gauss_blur(image, ksize=(5, 5), sigmaX=0):
    return cv2.GaussianBlur(image, ksize, sigmaX)



def add_color_overlay_to_lumen(image, overlay, alpha=0.3, color=(0, 255, 0)):
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    
    # Create the overlay_mask, where the overlay is color and everything else is black
    overlay_mask = np.zeros_like(image)
    overlay_mask[overlay == 255] = color   
    
    # add the overlay to the image
    cv2.addWeighted(overlay_mask, alpha, image, 1 - alpha, 0, image)
    
    print(f"image shape: {image.shape}")
    
    # return the image with the overlay
    return image


def add_membrane_overlay(image, overlay, color=(0, 0, 0)):
    """
    Show the original grayscale image wherever the overlay is white, and color (black) wherever the overlay is black.

    Args:
    - image: Grayscale input image (2D array).
    - overlay: Binary mask (2D array, same size as image).
    - color: Tuple representing the color to display where the overlay is black.

    Returns:
    - Image with the mask applied.
    """
    # Ensure the input image is grayscale and convert to BGR
    if len(image.shape) == 2:  # Check if image is grayscale
        image_bgr = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    else:
        image_bgr = image  # Already in BGR format

    # Create a mask for the areas to be blacked out (where overlay is 0)
    mask_inv = cv2.bitwise_not(overlay)

    # Convert the binary mask to a 3-channel BGR format
    mask_inv_bgr = cv2.cvtColor(mask_inv, cv2.COLOR_GRAY2BGR)

    # Create an image with the color (black by default) in areas where the overlay is black
    color_image = np.full_like(image_bgr, color, dtype=np.uint8)

    # Use the mask to combine the original image and the color image
    result = cv2.bitwise_and(image_bgr, mask_inv_bgr) + cv2.bitwise_and(color_image, cv2.cvtColor(overlay, cv2.COLOR_GRAY2BGR))

    return result

        
def get_raw_images(metadata_path, image_dir):
    
    metadata = pandas.read_csv(metadata_path)
    strip_filenames = metadata['strip_filename'].tolist()
    images = [image_dir + strip_filename for strip_filename in strip_filenames]
    
    return images


def remove_end_slash(path):
    # lambda to remove trailing slash from mask_output_dir if needed
    remove_end_slash = lambda path: path if not path.endswith("/") else path[:-1]

    return remove_end_slash(path)

def get_filename(base_path, suptitle, strip: str):
    print("get_filename() got strip = ", strip)
    output_dir = remove_end_slash(base_path)
    os.makedirs(output_dir, exist_ok=True)

    output_filename =  f"{output_dir}/{suptitle.replace(' ', '_').replace(',', '')}_strip_{strip}.png"

def save_membrane_mask(mask_output_directory, image_dict, suptitle, strip):
    """
    save_membrane_mask(image_dict, suptitle, image_dict['metadata']['strip'].values[0])
    
    Save the membrane mask (image_dict['morphed']) to a file, with a name that includes the suptitle and strip number.
    
    Ensures that the output directory exists.
    
    """
    
    mask_output_path = f"{mask_output_directory}/{suptitle.replace(' ', '_').replace(',', '')}"
    os.makedirs(mask_output_path, exist_ok=True)    
    cv2.imwrite(f"{mask_output_path}/strip_{strip}.png", image_dict['morphed'])


# function to take blur and non blur and create a plot of them
def plot_images(base_path:str, image_dict:dict, titles=None, display_image=True):
    if titles is None:
        titles = ["Original","Thresholded", "Morphed", "Overlayed"]
    suptitle = f"{image_dict['method']} thresholding, {'blurred' if image_dict['blur'] else 'not blurred'}, {'equalized' if image_dict['equalize'] else 'not equalized'}"
    images = [image_dict["image"], image_dict["thresholded"], image_dict["morphed"], image_dict["overlayed"]]
    fig, axes = plt.subplots(1, 4, figsize=(12, 5.5), facecolor='white')
    for i, ax in enumerate(axes):
        ax.imshow(images[i], cmap='gray')
        ax.axis('off')
        ax.set_title(titles[i])
    plt.tight_layout()
    plt.suptitle(suptitle, fontsize=16)

    # save just the membrane image, so we can use it later
    save_membrane_mask(f"{base_path}/masks", image_dict,suptitle, image_dict['metadata']['strip'].values[0])

    # save the comparison of raw and processed images step by step to a file in method_comparison
    comparison_output_path = f"{base_path}/method_comparison"
    print(f"comparison_output_path: {comparison_output_path}")
    os.makedirs(comparison_output_path, exist_ok=True)
    
    filename = f"{comparison_output_path}/{suptitle.replace(' ', '_').replace(',', '')}_strip_{image_dict['metadata']['strip'].values[0]}.png"
    print(f"Saving image to {filename}")    
    plt.savefig(filename)
    
    if display_image:
        plt.show()
    else:
        plt.close()
    

################### Thresholding methods to use #######################################
def adaptive_threshold(image, th_low=0, th_high=255):
    # Adaptive Mean Thresholding: Uses the mean of the neighborhood area to calculate the threshold.
    thresholded = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 25, 2)
    return thresholded

def adaptive_gaussian_threshold(image, th_low=0, th_high=255):
    # Adaptive Gaussian Thresholding: Uses the weighted sum of the neighborhood area to calculate the threshold.
    thresholded = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 25, 2)
    return thresholded

def otsu_threshold(image, th_low=0, th_high=255):
    # Otsu's Thresholding: Automatically calculates the optimal threshold value.
    _, thresholded = cv2.threshold(image, th_low, th_high, cv2.THRESH_OTSU)
    print(f"Otsu float: {_}")
    return thresholded

def simple_threshold(image, th_low=127, th_high=255):
    # Simple Thresholding: Applies a fixed threshold value to the image.
    _, thresholded = cv2.threshold(image, th_low, th_high, cv2.THRESH_BINARY)
    return thresholded

def trunc_threshold(image, th_low=127, th_high=255):
    # Truncation Thresholding: Pixels above the threshold are set to the threshold value.
    _, thresholded = cv2.threshold(image, th_low, th_high, cv2.THRESH_TRUNC)
    return thresholded

def tozero_threshold(image, th_low=127, th_high=255):
    # To Zero Thresholding: Pixels below the threshold are set to zero.
    # Can use this as a way to remove the background from the image, leaving only the foreground.
    _, thresholded = cv2.threshold(image, th_low, th_high, cv2.THRESH_TOZERO)
    return thresholded

def ostu_then_offset_threshold(image, th_low=127, th_high=255, offset=1.0):
    """ 
    Calcualte the Otsu threshold value, then add an offset to the threshold value.
    Use that newly offset value to re-threshold using a binary threshold.
    """
    
    otsu_th, thresholded = cv2.threshold(image, th_low, th_high, cv2.THRESH_OTSU)

    new_threshold = int(otsu_th * offset)
    print(f"Otsu threshold: {otsu_th}, new threshold: {new_threshold}")
    
    _, thresholded = cv2.threshold(image, new_threshold, th_high, cv2.THRESH_BINARY)
    
    return thresholded
    

########################### Morph Filter functions ######################################

def morph_close_then_open(image, morph_args: dict):
    """
        Perform opening and closing on the given image, using the specified kernels and iterations in the morph_args dictionary.
    """
    if (not "closing_kernel" in morph_args) or (not "opening_kernel" in morph_args) or (not "closing_iterations" in morph_args) or (not "opening_iterations" in morph_args):
        raise ValueError("Morphological arguments must include closing_kernel, opening_kernel, closing_iterations, and opening_iterations")
    
    closing1 = cv2.morphologyEx(image, cv2.MORPH_CLOSE, morph_args["closing_kernel"], iterations=morph_args["closing_iterations"])
    opening2 = cv2.morphologyEx(closing1, cv2.MORPH_OPEN, morph_args["opening_kernel"], iterations=morph_args["opening_iterations"])
    return opening2


########################################################################################

def process_image(image, method_key, equalize=True, blur=True, metadata=None, th_metadata=None):
    """ 
        This function takes an image and processes it using the specified method. Here 
        we need to define the helper methods we use for each of the steps.
    """
    
    print(f"Processing image with method_key {method_key}, th_metadata: {th_metadata}")
    
    th_low = th_metadata["th_low"]
    th_high = th_metadata["th_high"]
    threshold_func = th_metadata["threshold_func"]
    morph_filter_func = th_metadata["morph_filter_func"]
    threshold_name = th_metadata["threshold_name"]
    morph_filter_name = th_metadata["morph_filter_name"]
    
    # morphological filter parameters
    if equalize:
        image = equalize_image(image, clip_low=0, clip_high=255)
    if blur:
        image = gauss_blur(image, ksize=(5, 5), sigmaX=0)
        
    # threshold the image using the specified threshold function
    if threshold_func is None:
        raise ValueError("No threshold function provided")
    thresholded = threshold_func(image, th_low=th_low, th_high=th_high)
    
    # apply the morphological filter using the specified function
    if morph_filter_func is None:
        raise ValueError("No morphological filter function provided")
    morphed = morph_filter_func(thresholded, th_metadata["morph_args"])
    
    overlayed = add_membrane_overlay(image, morphed)
    
    return {
        "method": method_key,
        "blur": blur,
        "equalize": equalize,
        "metadata": metadata,
        "image": image,
        "thresholded": thresholded,
        "morphed": morphed,
        "overlayed": overlayed,
        "morph_filter_name": morph_filter_name,
        "threshold_name": threshold_name
    }

In [2]:
    # th_metadata = {
    # "otsu0": {
    #     "method": "otsu",
    #     "th_low": 0,
    #     "th_high": 255,
    #     "threshold_func": otsu_threshold,
    #     "threshold_name": "otsu_threshold",
    #     "morph_filter_func": morph_close_then_open,
    #     "morph_args": morph_args_close_open,
    #     "morph_filter_name": "custom_morph_filter"

In [4]:
# import the product function from itertools
def create_th_dict(threshold_key: str, morph_key: str, th_low: int, th_high: int,  threshold_func, morph_filter_func, morph_args: dict, offset_value: int = 1.0):
    return {
            "method": threshold_key,
            "morph_key": morph_key,
            "th_low": th_low,
            "th_high": th_high,
            "threshold_func": threshold_func,
            "threshold_name": f"{threshold_key}_{th_low}_{th_high}",
            "morph_filter_func": morph_filter_func,
            "morph_args": morph_args,
            "morph_filter_name": morph_key,
            "offset" : offset_value
    }

threshold_functions = {"adaptive": adaptive_threshold,
                        "adaptiveGaussian": adaptive_gaussian_threshold,
                        "otsu": otsu_threshold,
                        "simple": simple_threshold,
                        "trunc": trunc_threshold,
                        "tozero": tozero_threshold,
                        "ostuOffset": ostu_then_offset_threshold}

morph_filter_functions = {"closeOpen": morph_close_then_open}

morph_args = [{"type": "standard", "closing_kernel": np.ones((5, 5), np.uint8), "opening_kernel": np.ones((5, 5), np.uint8), "closing_iterations": 1, "opening_iterations": 1}]

# These are the combinations we want to test
morph_keys = [key for key in morph_filter_functions.keys()]
th_method_keys = [key for key in threshold_functions.keys()]

th_low_levels = [0, 70, 100]
th_high_levels = [130, 255]

offset_values = [0.8, 0.9, 1.0, 1.1, 1.2]

# create a list of all the combinations of the thresholding methods
all_combinations = product(th_method_keys, morph_keys, th_low_levels, th_high_levels, morph_args, offset_values)

# create a dictionary of dictionaries, where the key is the method_key+index
all_th_dicts = {}

    
for i, (th_key, morph_key, th_low, th_high, morph_arg, offset) in enumerate(all_combinations):
    th_dict = create_th_dict(th_key, morph_key, th_low, th_high, threshold_functions[th_key], morph_filter_functions[morph_key], morph_arg, offset)
    
    if th_dict is not None:
        all_th_dicts[f"{th_key}{i}"] = th_dict
    else:
        print(f"Skipping {th_key}{i}")
        

# print the first few keys
print(f"First few keys: {list(all_th_dicts.keys())[:5]}")